In [2]:
### Investing and Rebalincing a Portfolio

# 1) Instantiate
# Build a portfolio with the following target allocations:
# - Amazon(AMZN) = 40%
# - Cisco(CSCO) = 30%
# - General Electric(GE) = 30%

targets = {
    'AMZN': 0.40, # Amazon
    'CSCO': 0.30, # Cisco
    'GE': 0.30 # General Electric
}

In [6]:
import pandas as pd
import numpy as np

# Manually created data frame
portfolio = pd.DataFrame(
    index=list(targets.keys()) + ['CASH'],
    data={
        'date': '2020-09-25',
        'price': [np.NaN, np.NaN, np.NaN, 1],
        'target': [0.4, 0.3, 0.3, 0],
        'allocation': [0, 0, 0, 1],
        'shares': [0, 0, 0, 10000],
        'market_value': [0, 0, 0, 10000]
    }
)
print(portfolio)

            date  price  target  allocation  shares  market_value
AMZN  2020-09-25    NaN     0.4           0       0             0
CSCO  2020-09-25    NaN     0.3           0       0             0
GE    2020-09-25    NaN     0.3           0       0             0
CASH  2020-09-25    1.0     0.0           1   10000         10000


In [7]:
# Portfolio Instantiation Function
def instantiate_portfolio(targets, starting_balance):

    # Initialize CASH to zero.  This is not a security and does not 
    # need to be rebalanced.  It is just an overflow buffer
    targets['CASH'] = 0
    # Convert input dict into a list
    tickers = list(targets.keys())

    df = pd.DataFrame(
        index=tickers,
        columns=[
            'date','price','target',
            'allocation','shares','market_value'
        ]
    )
    df.shares = 0
    df.market_value = 0
    df.allocation = 0
    df.update(
        pd.DataFrame
            .from_dict(targets, orient="index")
            .rename(columns={0: 'target'})
    )
    df.at['CASH','shares'] = starting_balance

    return df

# Function Test
targets = {
    'AMZN': 0.40, # Amazon
    'CSCO': 0.30, # Cisco
    'GE': 0.30 # General Electric
}

portfolio = instantiate_portfolio(targets, 10000)
print(portfolio)

     date price target  allocation  shares  market_value
AMZN  NaN   NaN    0.4           0       0             0
CSCO  NaN   NaN    0.3           0       0             0
GE    NaN   NaN    0.3           0       0             0
CASH  NaN   NaN      0           0   10000             0


In [8]:
# Function to update ticker prices
def update_prices(portfolio, prices):
    prices['CASH'] = 1
    portfolio.update(pd.DataFrame({'price': prices}))
    portfolio.date = prices.name
    portfolio.market_value = portfolio.shares * portfolio.price

# fake for right now
prices = pd.Series(
    name='2018-01-01',
    data={'AMZN': 945.21, 'CSCO': 30.52, 'GE': 29.27}
)

print(prices)
update_prices(portfolio, prices)

AMZN    945.21
CSCO     30.52
GE       29.27
Name: 2018-01-01, dtype: float64


In [9]:
print(portfolio)

            date   price target  allocation  shares market_value
AMZN  2018-01-01  945.21    0.4           0       0            0
CSCO  2018-01-01   30.52    0.3           0       0            0
GE    2018-01-01   29.27    0.3           0       0            0
CASH  2018-01-01       1      0           0   10000        10000


In [10]:
# Orders will calculate the buy and sell orders to achieve target balances
# Does not edit the original portfolio.
def get_order(portfolio):
    total_value = portfolio.market_value.sum()

    order = (
        (total_value * portfolio.target // portfolio.price)  #// is a floor divisor to return quotiant
        - portfolio.shares
    ).drop('CASH')

    return order

# Use function to get buy amounts
order = get_order(portfolio)
print(order)

AMZN      4
CSCO     98
GE      102
dtype: object


In [11]:
# Deposit

def deposit(portfolio, amount):
    portfolio.at['CASH', 'shares'] += amount
    portfolio.at['CASH', 'market_value'] = portfolio.at['CASH','shares']

deposit(portfolio, 1000)

order = get_order(portfolio)
print(order)

AMZN      4
CSCO    108
GE      112
dtype: object


In [ ]:
# Simulate the execution of by/sell orders.

def simulate_process_order(portfolio,order):
    starting_cash = portfolio.at['CASH','SHARES']
    cash_adjustment = np.sum(order * portfolio.price)
    portfolio.shares += order
    portfolio.market_value = portfolio.shares * portfolio.price
    
